# Are you statistical suicidal?

The data science blog will use the kaggle data set __Suicide Rates Overview__ which compares socio-economic info with suicide rates by year and country (https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016).

Let's start by importing the CSV file into a __pandas.DataFrame__ and look at its first few rows.

In [30]:
import pandas as pd


# note that CSV file needs to be in the same location as your notebook
df = pd.read_csv('master.csv')

print(df.shape)
df.head()

(27820, 12)


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


First observations:

    - data set consists of 12 columns and 27820 rows
    - data looks well structured and consistent
    - columns are labeled with descriptive names (HDI: human development index, GDP: gross domestic product)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
country               27820 non-null object
year                  27820 non-null int64
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
country-year          27820 non-null object
HDI for year          8364 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


However, __pandas.DataFrame.info__ reveals several inconveniences regarding the column nomenclature, data types and missing values.

Taking a closer look 

    - column name 'gdp_for_year ($)' begins with a whitespace
    - white spaces are used as a seperator (e.g. HDI for year)
    - dollar signs are 
    
    

In [16]:
# deal with column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('_($)', '', regex=False)

# deal with false data type
df.gdp_for_year = df.gdp_for_year.str.replace(',', '').apply(int)

In [19]:
# deal with missing hdi values
hdi = pd.read_csv('Human Development Index (HDI).csv', skiprows=1)
hdi.dropna(axis=1, how='all', inplace=True)
hdi.Country = hdi.Country.str.strip()
hdi = hdi.melt(id_vars=['HDI Rank (2017)', 'Country'], var_name='year', value_name='value')
hdi.year = hdi.year.apply(int)
for i in range(0, len(df)):
    if pd.isnull(df.hdi_for_year.iloc[i]):
        v = hdi[(hdi.Country==df.country.iloc[i]) & (hdi.year==df.year.iloc[i])].value
        if len(v) > 0:
             df.at[i, 'hdi_for_year'] = v.values[0]
                
df.fillna(method='backfill', inplace=True)